In [1]:
from os import listdir
from os.path import isfile
from termcolor import cprint

from scipy.spatial import distance_matrix
import numpy as np

import ioutils 
import cplex_utils as cplex_utils

In [2]:
DATASET = 'AP'
INPUT_DIRECTORY = f"./data/{DATASET}/"

In [3]:
files = listdir(INPUT_DIRECTORY)
files.sort()
files = files[:4]
filter(isfile, files)
print(files)

for file in files:
    print ("Processing input " + file)
    N, p, alpha, delta, ksi, points, W = ioutils.parse_input(INPUT_DIRECTORY + file, DATASET)
    C = distance_matrix(points, points)
    
    M_1, X_1, H_1 = cplex_utils.get_umaphmp_model(N, p, alpha, delta, ksi, C, W, formulation='C', verbose=False)
    solution_1 = M_1.solve(log_output=False)
    Hubs_1 = [int(solution_1.get_var_value(H_1[i])) for i in range(N)]
    cost_1 = round(solution_1.objective_value, 6)

    M_2, X_2, Y_2, Z_2, H_2 = cplex_utils.get_umaphmp_model(N, p, alpha, delta, ksi, C, W, formulation='EK', verbose=False)
    solution_2 = M_2.solve(log_output=False)
    Hubs_2 = [int(solution_2.get_var_value(H_2[i])) for i in range(N)]
    cost_2 = round(solution_2.objective_value, 6)

    if Hubs_1 == Hubs_2:
        print ("["+ file + "] Same hubs found")
    else:
        cprint ("["+ file + "] Hubs differ: {} and {}!".format(Hubs_1, Hubs_2), "red")

    if cost_1 == cost_2:
        print ("["+ file + "] Same optimal cost")
    else:
        cprint ("["+ file + "] Costs differ: {} and {}!".format(cost_1, cost_2), "red")
    print ("----------------------------")

['10.2', '10.3', '10.4', '10.5']
Processing input 10.2


[10.2] Same hubs found
[10.2] Same optimal cost
----------------------------
Processing input 10.3
[10.3] Same hubs found
[10.3] Same optimal cost
----------------------------
Processing input 10.4
[10.4] Same hubs found
[10.4] Same optimal cost
----------------------------
Processing input 10.5
[10.5] Same hubs found
[10.5] Same optimal cost
----------------------------
